<a href="https://colab.research.google.com/github/zhangyingchengqi/Modern-Computer-Vision-with-PyTorch/blob/master/Chapter02/Specifying_batch_size_while_training_a_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第二章 第六节 batch size的调整测试

1. 什么是batch size?

在深度学习训练时，我们通常不会一次性把所有训练数据送进神经网络，而是**按批次（batch）**分成小块依次输入模型。

  * Batch Size = 每个批次里样本的数量

2. 为什么要用批量（Batch）？
  * 显存限制: 一次性输入太多数据会爆显存
  * 加速训练: 多个样本并行计算比逐个计算快
  * 梯度稳定性: Batch 的平均梯度比单样本梯度更稳定，训练更平滑
3. Batch Size 的取值?
  2 的幂次方（例如 16, 32, 64, 128, 256）: 因为 GPU 计算在 2 的幂时更高效
  * 太小（如 1~4） → 梯度波动大，训练慢
  * 太大（如 1024~8192） → 需要大量显存，可能欠拟合

In [2]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [3]:
x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

In [4]:
X = torch.tensor(x).float()
Y = torch.tensor(y).float()

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
X = X.to(device)
Y = Y.to(device)

In [7]:
class MyDataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x).float()
        self.y = torch.tensor(y).float()
    def __len__(self):
        return len(self.x)
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]
ds = MyDataset(X, Y)

/tmp/ipython-input-1110550737.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x).float()
/tmp/ipython-input-1110550737.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y).float()


In [9]:
dl = DataLoader(ds, batch_size=2, shuffle=True) #创建一个 数据加载器（DataLoader） 的，它的作用是帮你**按批次（batch）和顺序（是否打乱）**把数据集 ds 喂给模型

In [15]:
# 临时性测试一下上面的dl
for xb, yb in dl:
    print(xb, yb)  # xb 是批次的输入数据，yb 是批次的标签
# 把数据集 ds 按批次大小 2 切分，每个 epoch 前随机打乱顺序，然后让你可以一个 batch 一个 batch 地取出来训练

tensor([[1., 2.],
        [7., 8.]], device='cuda:0') tensor([[ 3.],
        [15.]], device='cuda:0')
tensor([[5., 6.],
        [3., 4.]], device='cuda:0') tensor([[11.],
        [ 7.]], device='cuda:0')


In [16]:
class MyNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_to_hidden_layer = nn.Linear(2,8)
        self.hidden_layer_activation = nn.ReLU()
        self.hidden_to_output_layer = nn.Linear(8,1)
    def forward(self, x):
        x = self.input_to_hidden_layer(x)
        x = self.hidden_layer_activation(x)
        x = self.hidden_to_output_layer(x)
        return x

In [18]:
mynet = MyNeuralNet().to(device)
loss_func = nn.MSELoss()
from torch.optim import SGD
opt = SGD(mynet.parameters(), lr = 0.001)

In [ ]:
import time
loss_history = []
start = time.time()
for _ in range(50):
    for data in dl:
        x, y = data
        opt.zero_grad()
        loss_value = loss_func(mynet(x),y)
        loss_value.backward()
        opt.step()
        loss_history.append(loss_value)
end = time.time()
print(end - start)

0.07826399803161621


In [ ]:
val_x = [[10,11]]

In [ ]:
val_x = torch.tensor(val_x).float().to(device)

In [ ]:
mynet(val_x)

tensor([[20.7331]], device='cuda:0', grad_fn=<AddmmBackward>)